In [1]:
import librosa
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import librosa, librosa.display
import os
import tensorflow as tf
print(tf.__version__)
import re
import plotly.express as px
import plotly.graph_objects as go
import sklearn
import datetime
import pandas as apd
import scipy, matplotlib.pyplot as plt, IPython.display as ipd
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, balanced_accuracy_score
import scipy
import math
import tqdm
import pickle

2.7.3


In [2]:
tf.random.set_seed(999)
np.random.seed(999)

In [3]:
# STFT_SPEC, STFT_MAG, MFCC, MEL_SPEC, CQT
dataset_prefix='STFT_MAG'

In [4]:
## librosa config
NUM_CLASSES=5
SAMPLE_RATE=16000
N_MFCC=40
WINDOW_LENGTH=512
OVERLAP=round(0.25 * WINDOW_LENGTH)
WINDOW = scipy.signal.hamming(WINDOW_LENGTH, sym=False)

In [5]:
basepath = './dataset/audio_wav'

In [6]:
original_df = pd.read_csv(os.path.join(basepath, 'linebot_data20220816.csv'))
# shuffle the data frame
original_df = original_df[['AudioFile_name', 'Feel_score', 'Feel_health']]
original_df = original_df.assign(AudioFile_name=original_df['AudioFile_name'].apply(lambda row: row + '.wav'))

print(len(original_df))
original_df = original_df[abs(original_df['Feel_score']-original_df['Feel_health'])<3]
print(len(original_df))

# original_df['Feel_score'] = np.where((original_df['Feel_score']==1) | (original_df['Feel_score']==2), 0, original_df['Feel_score'])
# original_df['Feel_score'] = np.where((original_df['Feel_score']==3), 1, original_df['Feel_score'])
# original_df['Feel_score'] = np.where((original_df['Feel_score']==4) | (original_df['Feel_score']==5), 2, original_df['Feel_score'])

# original_df['Feel_health'] = np.where((original_df['Feel_health']==1) | (original_df['Feel_health']==2), 0, original_df['Feel_health'])
# original_df['Feel_health'] = np.where((original_df['Feel_health']==3), 1, original_df['Feel_health'])
# original_df['Feel_health'] = np.where((original_df['Feel_health']==4) | (original_df['Feel_health']==5), 2, original_df['Feel_health'])

original_df = original_df.dropna().sample(frac=1).reset_index(drop=True)
original_df.head(10)

11883
11113


,AudioFile_name,Feel_score,Feel_health
0,Ue344acd77290079e5165c19140772cf8(2022-07-05 1...,2.0,4.0
1,U43c5c6b2b302a71069dbe5e12e2c1ff7(2022-05-25 2...,2.0,2.0
2,U9df7ed60efb0779c503d7fd148a02080(2022-05-23 1...,3.0,3.0
3,Ucd6cd370788775538664eb029e141821(2022-07-11 1...,3.0,3.0
4,Uff8c4b4497817e8e8fa794a106f75f9c(2022-07-12 1...,3.0,3.0
5,Uc3dbdca3deef87f6113250fc7d5b6ff2(2022-05-13 2...,2.0,1.0
6,U13e42e45d6287c63f72430d36edbe730(2022-05-24 0...,3.0,3.0
7,U7167942920ea4bd5f7249edcdb062efb(2022-06-13 0...,3.0,3.0
8,U8494d640671b2b7c9582a793f098c25e(2022-07-01 0...,1.0,1.0
9,U329b44a4e0293ac1c01b31f73b1c2a4b(2022-05-28 1...,3.0,3.0


In [13]:
g = original_df.groupby('Feel_score')
df1 = g.apply(lambda x: x.sample(g.size().min())).reset_index(drop=True)
df1

,AudioFile_name,Feel_score,Feel_health
0,U1ca586e7c3d486474012f4e77550eac3(2022-05-17 1...,1.0,1.0
1,U4485aa5166cf4ddfef47f62b9224ee32(2022-06-02 0...,1.0,1.0
2,U305abc97c823f7f55825a481d5446b13(2022-05-27 1...,1.0,1.0
3,U68984acdba658779621e2072c45a1d07(2022-06-25 2...,1.0,1.0
4,U7f7dbe6d121dd9d20a8dac89e656313e(2022-05-25 1...,1.0,1.0
...,...,...,...
1430,U32d95f90aac04388722047db1c1e9519(2022-05-22 0...,5.0,5.0
1431,Ue7990b536daa8a475d36e18ca950899f(2022-06-03 1...,5.0,5.0
1432,U4f578c6856af541792a31ee25d973add(2022-07-03 0...,5.0,5.0
1433,U5bf5273eb94dccdede5f5fd9ef509b06(2022-06-28 0...,5.0,5.0


In [14]:
g = original_df.groupby('Feel_health')
df2 = g.apply(lambda x: x.sample(g.size().min())).reset_index(drop=True)
df2

,AudioFile_name,Feel_score,Feel_health
0,U716f3681d442c3320de2cd6c2c86024a(2022-06-22 1...,1.0,1.0
1,Ub6a29db5853d9c2e78c9e5aa2d367bb8(2022-06-27 2...,3.0,1.0
2,U95c55a162cc2ff253e3a4b834402d0f9(2022-06-02 2...,2.0,1.0
3,U95c55a162cc2ff253e3a4b834402d0f9(2022-06-08 0...,2.0,1.0
4,U1ec34c20faa76a50df1ec15876e6b498(2022-05-19 1...,1.0,1.0
...,...,...,...
3010,U2e5c369bdc366b1759c6be741ef7f3c1(2022-06-11 1...,5.0,5.0
3011,U5bf5273eb94dccdede5f5fd9ef509b06(2022-08-05 2...,5.0,5.0
3012,U0f6976ce373367738f46619bf9827230(2022-06-29 1...,5.0,5.0
3013,U5bf5273eb94dccdede5f5fd9ef509b06(2022-07-26 0...,5.0,5.0


In [15]:
df = pd.concat([df1,df2]).drop_duplicates().sample(frac=1).reset_index(drop=True)
# df = df1.drop_duplicates().sample(frac=1).reset_index(drop=True)
df

,AudioFile_name,Feel_score,Feel_health
0,U716f3681d442c3320de2cd6c2c86024a(2022-07-03 1...,4.0,4.0
1,Ua2803ac3bbea6db27ad77b08bba722ad(2022-05-19 2...,3.0,2.0
2,U93a43aaeffd0cf73c8b26420bdc7609f(2022-05-22 2...,3.0,3.0
3,Uf5e382e2270dd9695e50f015d2c3c64c(2022-07-21 2...,3.0,2.0
4,Ufd06a3596bfb7af4b334780dcf70bf8b(2022-07-02 1...,5.0,5.0
...,...,...,...
3818,U716f3681d442c3320de2cd6c2c86024a(2022-06-21 1...,3.0,3.0
3819,U4f578c6856af541792a31ee25d973add(2022-05-18 1...,3.0,3.0
3820,Ua90416b44a9a58fbef172b7334b89013(2022-05-18 2...,2.0,1.0
3821,U985fa4d251a0b428e624aa3b0141dd08(2022-05-21 2...,4.0,5.0


In [16]:
print(df['AudioFile_name'][0], df.shape)

U716f3681d442c3320de2cd6c2c86024a(2022-07-03 10-57-30).wav (3823, 3)


In [19]:
fig = px.histogram(df['Feel_score'], x="Feel_score", nbins=5, 
                   color_discrete_sequence=['indianred'])
fig.update_layout(
    title_text='User Feel_score', # title of plot
    xaxis_title_text='Feel_score', # xaxis label
    yaxis_title_text='Count', # yaxis label
    bargap=0.01, # gap between bars of adjacent location coordinates
    bargroupgap=0.1 # gap between bars of the same location coordinates
)

fig.show()

In [20]:
fig = px.histogram(df['Feel_health'], x="Feel_health", nbins=5, 
                   color_discrete_sequence=['indianred'])
fig.update_layout(
    title_text='User Feel_health', # title of plot
    xaxis_title_text='Feel_health', # xaxis label
    yaxis_title_text='Count', # yaxis label
    bargap=0.01, # gap between bars of adjacent location coordinates
    bargroupgap=0.1 # gap between bars of the same location coordinates
)

fig.show()

In [21]:
score_cnt = np.bincount(df['Feel_score'])[1:]
for i, cnt in enumerate(score_cnt):
    print(f"Feel_score: {i+1}, Count: {cnt}")
health_cnt = np.bincount(df['Feel_health'])[1:]
for i, cnt in enumerate(health_cnt):
    print(f"Feel_health: {i+1}, Count: {cnt}")

Feel_score: 1, Count: 287
Feel_score: 2, Count: 824
Feel_score: 3, Count: 1331
Feel_score: 4, Count: 738
Feel_score: 5, Count: 643
Feel_health: 1, Count: 603
Feel_health: 2, Count: 766
Feel_health: 3, Count: 851
Feel_health: 4, Count: 851
Feel_health: 5, Count: 752


In [22]:
class FeatureExtractor:
    def __init__(self, windowLength, overlap, sample_rate):
        self.ffT_length = windowLength
        self.window_length = windowLength
        self.overlap = overlap
        self.sample_rate = sample_rate
        self.window = scipy.signal.hamming(self.window_length, sym=False)

    def get_stft_spectrogram(self, audio, spectrogram=False):
        S = librosa.stft(audio, n_fft=self.ffT_length, win_length=self.window_length, hop_length=self.overlap,
                            window=self.window, center=True)
        S = np.abs(S)
        if spectrogram:
            return np.square(S)
        else:
            return S

    def get_constant_q_spectrogram(self, audio, n_bins=84):
        # Compute the constant-Q transform of an audio signal.
        C = librosa.core.cqt(audio, sr=self.sample_rate, hop_length=self.overlap, fmin=None,
                                n_bins=n_bins, bins_per_octave=12, tuning=0.0,
                                filter_scale=1, norm=1, sparsity=0.01, window='hann', scale=True,
                                pad_mode='reflect',
                                res_type=None)
        return np.abs(C)

    def get_mfcc_features(self, audio, n_mfcc=40):
        S = self.get_mel_spectrogram(audio)
        return librosa.feature.mfcc(S=librosa.power_to_db(S), sr=self.sample_rate, n_mfcc=n_mfcc)

    def get_mel_spectrogram(self, audio):
        D = self.get_stft_spectrogram(audio, spectrogram=True)
        S = librosa.feature.melspectrogram(S=D, sr=self.sample_rate, power=2.0, pad_mode='reflect',
                                              n_fft=self.ffT_length, hop_length=self.overlap, center=True)
        return S

In [23]:
extractor = FeatureExtractor(WINDOW_LENGTH, OVERLAP, SAMPLE_RATE)

In [24]:
def get_feature_extractor_func(feature_type):
    print("Feature type:", feature_type)
    feature_map = {'STFT_SPEC': lambda audio: extractor.get_stft_spectrogram(audio, spectrogram=True),
                   'STFT_MAG' : lambda audio: extractor.get_stft_spectrogram(audio, spectrogram=False),
                   'MFCC': lambda audio: extractor.get_mfcc_features(audio),
                   'MEL_SPEC': lambda audio: extractor.get_mel_spectrogram(audio),
                   'CQT': lambda audio: extractor.get_constant_q_spectrogram(audio)}
    return feature_map.get(feature_type)

In [25]:
get_feature_vector = get_feature_extractor_func(dataset_prefix)

Feature type: STFT_MAG


In [26]:
max_feature_len = []
def extract_features(audio, sr, max_pad_len):
    try:
        features = get_feature_vector(audio)
        pad_width = max_pad_len - features.shape[1]
        max_feature_len.append(features.shape[1])
        
        if pad_width < 0:
#             print("Cropping original features depth of:", features.shape[1], 'to:', max_pad_len)
            features = features[:,:max_pad_len]
        else:
            features = np.pad(features, pad_width=((0, 0), (0, pad_width)), mode='wrap')
            
    except Exception as e:
        print("Error encountered.", e)
        return None 
     
    return features

def _remove_silent_frames(audio):
    trimed_audio = []
    indices = librosa.effects.split(audio, top_db=60)

    for index in indices:
        trimed_audio.extend(audio[index[0]: index[1]])
    return np.array(trimed_audio)

In [27]:
def _audio_random_crop(audio, duration, sample_rate):
    # Compute the duration (in seconds) of an audio time series
    audio_duration_secs = librosa.core.get_duration(audio, sample_rate)

    ## duration: length of the cropped audio in seconds
    if duration >= audio_duration_secs:
        print("Passed duration greater than audio duration of: ", audio_duration_secs)
        audio = np.pad(audio, pad_width=((0, 0), (0, audio.size)), mode='wrap')
        return audio

    audio_duration_ms = math.floor(audio_duration_secs * sample_rate)
    duration_ms = math.floor(duration * sample_rate)
    idx = np.random.randint(0, audio_duration_ms - duration_ms)
    return audio[idx: idx + duration_ms]

In [28]:
def next_record(df):
    for index, row in df.iterrows():
        filename = row['AudioFile_name']
        audio_path = os.path.join(basepath, filename)
        audio, sr = librosa.load(audio_path, sr=SAMPLE_RATE)
        audio =  _remove_silent_frames(audio)
        # using mfccs and random cropping the audio produces high overfitting
        # audio = _audio_random_crop(audio, 1.6, SAMPLE_RATE)
        
        # for mfcc, crop to 311
        # for CQT, STFT and MEL, crop to 1241
        
        feature_vector = extract_features(audio, sr, 1241)
        feel_score = row['Feel_score']
        feel_health = row['Feel_health']
        yield feature_vector, feel_score, feel_health, filename

In [29]:
features = []
feel_scores = []
feel_healths = []
filenames = []

print(len(df))
i = 0
data_generator = next_record(df)
for input_, feel_score, feel_health, filename in data_generator:
    features.append(input_)
    feel_scores.append(feel_score)
    feel_healths.append(feel_health)
    filenames.append(filename)
    i += 1
    if i%500==0:
        print(f"dataset prepared: {i}")
print(features[0].shape)

3823
dataset prepared: 500
dataset prepared: 1000
dataset prepared: 1500
dataset prepared: 2000
dataset prepared: 2500
dataset prepared: 3000
dataset prepared: 3500
(257, 1241)


In [30]:
filenames = np.asarray(filenames)
features = np.asarray(features)
feel_scores = np.asarray(feel_scores)
feel_healths = np.asarray(feel_healths)
features = np.expand_dims(features, axis=3)

In [31]:
# normalize the score to the range [0-1]
feel_scores = (feel_scores - 1) / 4
print(np.min(feel_scores), np.max(feel_scores))
print(feel_scores[:20])
feel_healths = (feel_healths - 1) / 4
print(np.min(feel_healths), np.max(feel_healths))
print(feel_healths[:20])

0.0 1.0
[0.75 0.5  0.5  0.5  1.   0.5  0.5  0.25 0.5  0.   0.75 0.75 0.75 0.5
 1.   0.75 1.   0.25 0.25 0.25]
0.0 1.0
[0.75 0.25 0.5  0.25 1.   0.5  0.75 0.75 0.25 0.   0.5  0.75 0.75 0.25
 1.   0.75 1.   0.   0.25 0.25]


In [32]:
print("Feature shape:", features.shape)
print("Feature range (Min/Max):", np.min(max_feature_len), np.max(max_feature_len))

Feature shape: (3823, 257, 1241, 1)
Feature range (Min/Max): 149 7629


In [33]:
print("Training data shape")
print(features.shape)
print(feel_scores.shape)
print(feel_healths.shape)

Training data shape
(3823, 257, 1241, 1)
(3823,)
(3823,)


In [34]:
# train test split
test_size_percentage = 0.2
test_size = int(test_size_percentage * features.shape[0])

In [35]:
X_train = {'filenames': filenames[:-test_size], 
           'features': features[:-test_size],
           'feel_scores': feel_scores[:-test_size],
           'feel_healths': feel_healths[:-test_size]}

X_test = { 'filenames': filenames[-test_size:], 
           'features': features[-test_size:],
           'feel_scores': feel_scores[-test_size:],
           'feel_healths': feel_healths[-test_size:]}

In [36]:
with open('./dataset/' + dataset_prefix + 'linebot_train' + str(WINDOW_LENGTH) + '_balanced.pickle', 'wb') as handle:
    pickle.dump(X_train, handle, protocol=4)
with open('./dataset/' + dataset_prefix + 'linebot_test' + str(WINDOW_LENGTH) + '_balanced.pickle', 'wb') as handle:
    pickle.dump(X_test, handle, protocol=4)
# np.save('./dataset/' + dataset_prefix + 'linebot_train' + str(WINDOW_LENGTH), X_train)
# np.save('./dataset/' + dataset_prefix + 'linebot_test' + str(WINDOW_LENGTH), X_test)